<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/097_Research_Summarizer_Agent_Dirty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For a **Research Summarizer Agent** that’s focused on summarizing key points of text docs, we’ll want at least these capabilities in the **Environment** so the Agent can fully accomplish the goal:

---

### **Actions / Tools Needed**

1. **`list_txt_files`** – List all `.txt` documents in `/content/files`
2. **`read_txt_file`** – Read the content of a specified file
3. **`summarize_text`** – Summarize a given chunk of text (this will call the LLM)
4. **`write_summary_file`** – Save the generated summary into `/content/summaries`
5. *(Optional)* **`search_text_in_file`** – If you want the Agent to pull specific sections before summarizing

---

### **Proposed GOAL Object**

```python
file_summary_goal = Goal(
    priority=1,
    name="file_summary",
    description="""
    Summarize the key points of text documents in /content/files.
    Steps:
    1. List all available text files using list_txt_files.
    2. Read the content of each relevant file with read_txt_file.
    3. Create a concise, clear summary highlighting the most important points.
    4. Save the summary to /content/summaries using write_summary_file.
    """
)
```

---

### **Workflow in English**

* The **Agent** starts with the **Goal** of summarizing all text files in the `/content/files` folder.
* It calls `list_txt_files` to see what’s available.
* For each file, it calls `read_txt_file` to fetch the contents.
* It sends the content to `summarize_text` (LLM-based) to condense into a summary.
* Finally, it calls `write_summary_file` to save the result in `/content/summaries`.




In [ ]:
# Research Summarizer — Environment & Actions
# - Local text file environment (list/read/write)
# - LLM-powered summarize_text action (uses OpenAI via a small helper)
# - ActionRegistry wiring with JSON-schema-like parameter specs

from typing import Dict, Any, List, Optional, Callable
import os, io, time, traceback, re

# --- Base Environment from your template (kept for consistency) ---
class Environment:
    def execute_action(self, action, args: Dict[str, Any]) -> Dict[str, Any]:
        try:
            result = action.execute(**args)
            return {"tool_executed": True, "result": result, "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")}
        except Exception as e:
            return {
                "tool_executed": False,
                "error": str(e),
                "traceback": traceback.format_exc(),
                "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")
            }

# --- Actions & Registry (same interface as your template) ---
class Action:
    def __init__(self, name: str, fn: Callable, description: str, parameters: Dict, terminal: bool=False):
        self.name, self.fn, self.description = name, fn, description
        self.parameters, self.terminal = parameters, terminal
    def execute(self, **kwargs):
        return self.fn(**kwargs)

class ActionRegistry:
    def __init__(self):
        self._actions: Dict[str, Action] = {}
    def register(self, action: Action):
        if action.name in self._actions:
            raise ValueError(f"Action already registered: {action.name}")
        self._actions[action.name] = action
    def get_action(self, name: str) -> Optional[Action]:
        return self._actions.get(name)
    def get_actions(self) -> List[Action]:
        return list(self._actions.values())
    def validate_args(self, action: Action, args: Dict[str, Any]) -> (bool, str):
        schema = action.parameters or {"type":"object","properties":{},"required":[]}
        for key in schema.get("required", []):
            if key not in args:
                return False, f"Missing required arg: {key}"
        return True, "ok"


# --- Research Summarizer Environment ---
class ResearchEnvironment(Environment):
    base_dir = "/content/files"
    out_dir  = "/content/summaries"

    @staticmethod
    def _safe_join(base: str, name: str) -> str:
        path = os.path.abspath(os.path.join(base, name))
        if not path.startswith(os.path.abspath(base)):
            raise ValueError("Invalid path")
        return path

    @staticmethod
    def _sanitize(name: str) -> str:
        stem = os.path.splitext(os.path.basename(name))[0]
        safe = re.sub(r"[^a-zA-Z0-9._-]", "_", stem)
        return safe + ".summary.txt"

    def list_txt_files(self) -> List[str]:
        if not os.path.exists(self.base_dir):
            return []
        return sorted([f for f in os.listdir(self.base_dir) if f.lower().endswith('.txt')])

    def read_txt_file(self, file_name: str) -> Dict[str, Any]:
        path = self._safe_join(self.base_dir, file_name)
        if not os.path.exists(path):
            return {"error": f"File not found: {file_name}", "hint": "Call list_txt_files first"}
        with io.open(path, 'r', encoding='utf-8', errors='replace') as f:
            text = f.read()
        # Truncate very large documents for safety; agent can iterate in chunks if needed
        max_chars = 12000
        truncated = len(text) > max_chars
        if truncated:
            text = text[:max_chars] + "\n... [truncated]"
        return {"file_name": file_name, "content": text, "truncated": truncated}

    def write_summary_file(self, source_file: str, content: str) -> str:
        os.makedirs(self.out_dir, exist_ok=True)
        out_name = self._sanitize(source_file)
        out_path = self._safe_join(self.out_dir, out_name)
        with io.open(out_path, 'w', encoding='utf-8') as f:
            f.write(content)
        return out_path


### What’s the “base Environment”?

Think of it as the **body’s nervous system**: a tiny, reusable wrapper that **executes any Action safely** and **returns a uniform result envelope**.

```python
class Environment:
    def execute_action(self, action, args: Dict[str, Any]) -> Dict[str, Any]:
        try:
            result = action.execute(**args)
            return {"tool_executed": True, "result": result, "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")}
        except Exception as e:
            return {"tool_executed": False, "error": str(e), "traceback": traceback.format_exc(),
                    "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")}
```

#### Why this exists

* **Uniform envelope**: The agent never has to guess what a tool returns.
* **Centralized safety**: Errors are caught and converted into structured feedback the LLM can recover from.
* **Reusability**: Works for any domain—files, APIs, DBs, etc.

---

### What’s the “ResearchEnvironment” then?

That’s a **specialized Environment** with **domain-specific operations** for your use case (local text summarization). It provides concrete, safe methods like:

* `list_txt_files()` — list sources
* `read_txt_file(file_name)` — read a doc with path checks + truncation guard
* `write_summary_file(source_file, content)` — save a summary with safe filenames

It still **inherits the base behavior** (uniform envelopes via `execute_action`) but adds the **actual tools** this agent needs.

A sketch:

```python
class ResearchEnvironment(Environment):
    base_dir = "/content/files"
    out_dir  = "/content/summaries"

    def list_txt_files(self):
        return sorted(f for f in os.listdir(self.base_dir) if f.lower().endswith(".txt"))

    def read_txt_file(self, file_name: str):
        path = os.path.abspath(os.path.join(self.base_dir, file_name))
        if not path.startswith(os.path.abspath(self.base_dir)) or not os.path.exists(path):
            return {"error": f"File not found: {file_name}", "hint": "Call list_txt_files first"}
        with io.open(path, "r", encoding="utf-8", errors="replace") as f:
            text = f.read()
        if len(text) > 12000:
            text = text[:12000] + "\n... [truncated]"
        return {"file_name": file_name, "content": text}

    def write_summary_file(self, source_file: str, content: str):
        os.makedirs(self.out_dir, exist_ok=True)
        safe = re.sub(r"[^a-zA-Z0-9._-]", "_", os.path.splitext(os.path.basename(source_file))[0])
        out_path = os.path.join(self.out_dir, f"{safe}.summary.txt")
        with io.open(out_path, "w", encoding="utf-8") as f:
            f.write(content)
        return out_path
```

---

### Why have both layers?

* **Base Environment = generic executor**
  Keeps the orchestrator simple: it always calls `environment.execute_action(action, args)` and gets a structured reply.
* **Domain Environment = concrete tools**
  Encapsulates *how* to interact with your world (files, paths, truncation, naming). If you later switch from local files to, say, S3 or Github, you **swap this class**, not the orchestrator or actions logic.

---

### Where do `Action` and `ActionRegistry` fit?

* `Action` wraps a callable (e.g., `env.read_txt_file`) with metadata (description, JSON-like schema).
* `ActionRegistry` is the **tool shed**—you **register** these actions so the agent can discover and invoke them by name.
  Example:

  ```python
  registry.register(Action(
      name="read_txt_file",
      fn=lambda file_name: env.read_txt_file(file_name),
      description="Read a text file from /content/files",
      parameters={"type":"object","properties":{"file_name":{"type":"string"}}, "required":["file_name"]}
  ))
  ```

---

### TL;DR

* Keep the **Base Environment**: generic, safe execution + uniform envelopes.
* Add a **ResearchEnvironment**: your domain-specific “body” with file/list/read/write logic.
* Register env methods as **Actions** in the **ActionRegistry** so the orchestrator can call them via tool selection.

If you’re ready, the next step is: **define `ResearchEnvironment`**, then **register its methods as actions**. After that we’ll plug them into the orchestrator.


In [ ]:
# --- LLM Summarization Helper (uses your OpenAI client outside the loop) ---
def make_summarizer(openai_chat_fn: Callable[[List[Dict[str, str]]], str]):
    """Return a summarize_text(text, max_points, style) function using provided LLM call.
    openai_chat_fn: function that takes messages=[...] and returns string content.
    """
    def summarize_text(text: str, max_points: int = 5, style: str = "bullet") -> str:
        system = (
            "You are a precise technical summarizer. Extract key points, preserve facts, "
            "and avoid speculation. Keep it concise."
        )
        user = (
            f"Summarize the following text into at most {max_points} key points. "
            f"Format: {'bullets' if style=='bullet' else 'short paragraphs'}.\n\n" + text
        )
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ]
        return openai_chat_fn(messages)
    return summarize_text

# --- Wiring helper to build the registry for this agent ---
def build_research_actions(env: ResearchEnvironment, summarizer_fn: Callable[[str, int, str], str]) -> ActionRegistry:
    registry = ActionRegistry()

    registry.register(Action(
        name="list_txt_files",
        fn=lambda: env.list_txt_files(),
        description="Return .txt file names from /content/files",
        parameters={"type":"object","properties":{},"required":[]},
    ))

    registry.register(Action(
        name="read_txt_file",
        fn=lambda file_name: env.read_txt_file(file_name),
        description="Read a text file from /content/files",
        parameters={
            "type": "object",
            "properties": {"file_name": {"type": "string"}},
            "required": ["file_name"],
        },
    ))

    registry.register(Action(
        name="summarize_text",
        fn=lambda text, max_points=5, style="bullet": summarizer_fn(text, max_points, style),
        description="Summarize raw text into key points using the LLM",
        parameters={
            "type": "object",
            "properties": {
                "text": {"type": "string"},
                "max_points": {"type": "integer", "minimum": 1, "maximum": 12},
                "style": {"type": "string", "enum": ["bullet", "paragraph"]},
            },
            "required": ["text"],
        },
    ))

    registry.register(Action(
        name="write_summary_file",
        fn=lambda source_file, content: env.write_summary_file(source_file, content),
        description="Write summary text to /content/summaries (auto-named from source)",
        parameters={
            "type": "object",
            "properties": {
                "source_file": {"type": "string"},
                "content": {"type": "string"},
            },
            "required": ["source_file", "content"],
        },
    ))

    return registry

## Dependency Injection

Instead of your code **creating its own dependencies inside** (hard-coding them),
you **pass them in from the outside** so they can be swapped, mocked, or upgraded without touching the main logic.

---

### Example without dependency injection (hard-coded dependency)

```python
def summarize_text(text):
    # Directly calls OpenAI here
    return openai_client.chat(messages=[{"role": "user", "content": text}])
```

**Problems:**

* Can’t test without hitting the API.
* Stuck with `openai_client` — can’t swap to Anthropic or local model without editing this function.
* Harder to reuse in other projects.

---

### Example **with** dependency injection

```python
def make_summarizer(chat_fn):
    def summarize_text(text):
        return chat_fn([{"role": "user", "content": text}])
    return summarize_text

# Pass in whatever LLM you want
summarizer = make_summarizer(openai_client.chat)
```

**Benefits:**

* In tests, you can do:

  ```python
  fake_summarizer = make_summarizer(lambda msgs: "fake summary")
  ```
* In prod, you can do:

  ```python
  real_summarizer = make_summarizer(openai_client.chat)
  ```
* Main summarizer logic doesn’t care *which* LLM it’s using — it just calls the function it was given.

---

💡 In your code,
`make_summarizer(openai_chat_fn)` is doing **dependency injection**:
the *dependency* = the OpenAI chat function,
and you’re *injecting* it into the summarizer logic instead of hard-coding it inside.

---

## What to focus on

### 1) `make_summarizer(...)` = dependency injection

* You pass in `openai_chat_fn(messages) -> str`, and it returns a ready-to-call `summarize_text(text, max_points, style)`.
* This **decouples** your environment/actions from any specific LLM client. In tests you can pass a mock; in prod you pass your real OpenAI caller.
* The inner function is **pure** (no state), so it’s easy to reason about and reuse.

### 2) Strong, simple system prompt

* The `system` text sets behavior tightly: “precise technical summarizer… preserve facts…”
* This reduces model wandering and keeps summaries consistent.

### 3) Explicit knobs for the LLM (`max_points`, `style`)

* Clear, bounded control surface for the model output.
* `style` acts like a formatting contract (bullets vs short paragraphs).
* These are easy to expose as tool args later if you want the agent to pick them.

### 4) ActionRegistry wiring = clean “tool shed”

Each action entry has:

* **`name`**: specific and human-readable.
* **`fn`**: a tiny lambda that calls the real implementation (`env.*` or `summarizer_fn`).
* **`description`**: tells the LLM when to use it.
* **`parameters` (JSON-schema-like)**: defines what args are allowed/required.

  * This is huge for reliability; your orchestrator can validate before execution.

### 5) Constrained schemas to reduce errors

* `read_txt_file` requires `"file_name"`.
* `summarize_text` constrains `max_points` (1–12) and `style` (enum).
* Tight schemas = fewer tool-call mistakes and better LLM focus.

### 6) No hidden side effects in the tools

* `list_txt_files` and `read_txt_file` are **read-only**.
* `write_summary_file` is the only writer, and it names outputs safely in one place.
* This makes behavior predictable and easier to audit.

### 7) Small lambdas, big clarity

* The lambdas keep the registry lightweight; the real logic lives in `env` or `summarizer_fn`.
* You can swap `env` (local → S3 → GitHub) without changing the registry interface.

# Nice upgrades (if/when you need them)

* **Chunking for long docs:** add a `read_txt_file_chunked(file_name, start, n_chars)` tool so the agent can iterate large texts.
* **Deterministic truncation signal:** your `read_txt_file` already returns `truncated`; good. The agent can branch on it.
* **Terminate tool:** register a `terminate` action (`terminal=True`) so the agent can end cleanly when summaries are saved.
* **Return schema** (optional): some teams add a `return_schema` in `parameters` or Action metadata to validate outputs post-exec.

# Quick checklist for this block

* [x] Summarizer is **injected** via `make_summarizer` (mockable).
* [x] Actions have **clear names**, **good descriptions**, **tight schemas**.
* [x] Only one tool writes to disk; all others are read-only.
* [x] Parameters constrain LLM behavior (`enum`, `min/max`).

If you’re happy with this, next step is to define `ResearchEnvironment` (you likely already have it) and then wire these actions + summarizer into your orchestrator. After that we’ll run an end-to-end pass.


In [ ]:
# STEP 2 — Language & Prereqs (clean)
# Put this ABOVE the wiring cell. Defines: Goal, Memory, AgentLanguage, SummarizerLanguage.
from dataclasses import dataclass
from typing import List, Dict, Any

# --- Minimal prereqs --------------------------------------------------------
@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str

class Memory:
    def __init__(self):
        self.items: List[Dict[str, Any]] = []
    def add_memory(self, m: Dict[str, Any]):
        self.items.append(m)
    def get_memories(self, limit: int | None = None):
        return self.items[-limit:] if limit else self.items

# --- AgentLanguage base + concrete SummarizerLanguage ----------------------
class AgentLanguage:
    """Build prompt for the LLM; parse the LLM's response (usually handled by generate_response)."""
    def construct_prompt(self, actions: List[Any], environment: Any, goals: List[Goal], memory: Memory) -> Dict[str, Any]:
        raise NotImplementedError
    def parse_response(self, response: Dict[str, Any]) -> Dict[str, Any]:
        # Default: response already structured as {"tool": ..., "args": {...}}
        return response

class SummarizerLanguage(AgentLanguage):
    """Formats goals/memory for the summarizer agent. Tool-call parsing is done in generate_response()."""
    def construct_prompt(self, actions, environment, goals: List[Goal], memory: Memory) -> Dict[str, Any]:
        goals_text = (
            "You are a file summarizer. Follow these goals in order of priority:\n" +
            "\n".join(f"- ({g.priority}) {g.name}: {g.description.strip()}" for g in sorted(goals, key=lambda g: g.priority))
        )
        mem = memory.get_memories(8)
        return {"goals_text": goals_text, "memory": mem, "actions": actions}



In [ ]:
!pip -q install openai python-dotenv

That block is the **wiring layer** that lets the LLM pick tools via **function calling**. It’s good—and it belongs **after** you’ve defined:

* `Agent` (orchestrator), `Goal`, `Action`, `ActionRegistry`, `Memory`, `Environment`
* `ResearchEnvironment`, `make_summarizer`, `build_research_actions`
* Base `AgentLanguage` (your `SummarizerLanguage` subclasses it)

### What it does

* Exports your registry to OpenAI’s `tools` format (`registry_to_openai_tools`)
* Builds a compact prompt (`SummarizerLanguage.construct_prompt`)
* Calls `gpt-4o-mini` with `tools=...` and parses the **tool call** into `{ "tool": ..., "args": ... }`
* If no tool call is returned, it safely defaults to `list_txt_files`

### Things to double-check

* Make sure **base `AgentLanguage`** is defined earlier (or include a small base class).
* Ensure `ResearchEnvironment`, `make_summarizer`, and `build_research_actions` are already loaded.
* Your `.env` path and key name: `'/content/API_KEYS.env'` with `OPENAI_API_KEY`.








In [ ]:
# Research Summarizer — Orchestrator Wiring (function calling)
# REQUIREMENTS (already defined earlier in your notebook):
# - Agent (orchestrator template)
# - Goal, Action, ActionRegistry
# - ResearchEnvironment, make_summarizer, build_research_actions (from the previous cell)
# - Memory class from your template
#
# This cell wires those pieces together, adds an AgentLanguage
# and a generate_response() that uses OpenAI function calling.

import os, json
from typing import Dict, Any, List
from dotenv import load_dotenv
from openai import OpenAI

# ---------------- Load API key & client ----------------
load_dotenv('/content/API_KEYS.env')
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL = "gpt-4o-mini"

# ---------------- Tools export helper ------------------
def registry_to_openai_tools(registry: ActionRegistry) -> List[Dict[str, Any]]:
    tools = []
    for a in registry.get_actions():
        tools.append({
            "type": "function",
            "function": {
                "name": a.name,
                "description": a.description,
                "parameters": a.parameters or {"type": "object", "properties": {}, "required": []},
            },
        })
    return tools

# ---------------- AgentLanguage ------------------------
class SummarizerLanguage(AgentLanguage):
    """Formats goals/memory for the LLM; parse is handled in generate_response."""
    def construct_prompt(self, actions, environment, goals, memory):
        goals_text = "You are a file summarizer. Follow these goals in order of priority:\n" + "\n".join(
            f"- ({g.priority}) {g.name}: {g.description.strip()}" for g in sorted(goals, key=lambda g: g.priority)
        )
        # Keep a tight memory window
        mem = memory.get_memories(8)
        return {"goals_text": goals_text, "memory": mem, "actions": actions}

# ------------- generate_response (OpenAI call) ---------
# NOTE: This returns a structured dict {"tool": name, "args": {...}} for the orchestrator.

def make_generate_response(registry: ActionRegistry):
    tools_spec = registry_to_openai_tools(registry)

    def build_messages(prompt_dict: Dict[str, Any]) -> List[Dict[str, str]]:
        system = (
            prompt_dict["goals_text"]
            + "\n\nYou must use tools via function calling to make progress. "
            + "Choose exactly one next tool per step. If you have saved all summaries, call a terminate tool if available; otherwise indicate completion."
        )
        # Replay memory if you'd like the model to see prior context (optional here)
        memory_msgs = []
        for m in prompt_dict["memory"]:
            role = m.get("role") or m.get("type") or "user"
            content = m.get("content")
            # Coerce non-strings for safety
            if not isinstance(content, str):
                content = json.dumps(content)
            memory_msgs.append({"role": role if role in ("system","user","assistant") else "user", "content": content})

        # Nudge the model with a fresh user instruction
        user_msg = {
            "role": "user",
            "content": (
                "Pick the best next tool from the available functions to progress toward summarizing the files. "
                "Return a function call, not prose."
            ),
        }
        return [{"role": "system", "content": system}] + memory_msgs + [user_msg]

    def _generate_response(prompt_dict: Dict[str, Any]) -> Dict[str, Any]:
        messages = build_messages(prompt_dict)
        resp = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools_spec,
            tool_choice="auto",
            temperature=0.2,
        )
        msg = resp.choices[0].message
        # If the model chose a tool, parse it
        if msg.tool_calls:
            call = msg.tool_calls[0]
            name = call.function.name
            try:
                args = json.loads(call.function.arguments or "{}")
            except json.JSONDecodeError:
                args = {}
            return {"tool": name, "args": args}
        # Fallback if no tool was called; gently kick off with list_txt_files
        return {"tool": "list_txt_files", "args": {}}

    return _generate_response

# ---------------- Build environment & tools -------------
env = ResearchEnvironment()

# Summarizer uses your OpenAI client under the hood

def openai_chat_fn(messages):
    resp = client.chat.completions.create(model=MODEL, messages=messages)
    return resp.choices[0].message.content

summarizer = make_summarizer(openai_chat_fn)
registry = build_research_actions(env, summarizer)

# ---------------- Goals --------------------------------
file_summary_goal = Goal(
    priority=1,
    name="file_summary",
    description=(
        "Summarize key points of text documents in /content/files.\n"
        "Steps: 1) list files, 2) read each file, 3) summarize to ≤5 bullets, 4) write to /content/summaries."
    ),
)

# ---------------- Orchestrator instance -----------------
language = SummarizerLanguage()
generate_response = make_generate_response(registry)

agent = Agent(
    goals=[file_summary_goal],
    agent_language=language,
    action_registry=registry,
    generate_response=generate_response,
    environment=env,
)

This “STEP 1 — Base Orchestrator (GAME skeleton)” block is exactly the **foundational layer** you should run first. Here’s how it fits and what to watch for:

# Where this block fits

* It defines your **core GAME building blocks**:

  * **G**: `Goal`
  * **A**: `Action`, `ActionRegistry` (+ lightweight `validate_args`)
  * **M**: `Memory`
  * **E**: `Environment` (base, uniform result envelope)
* Everything else (ResearchEnvironment, summarizer tool, wiring, function-calling, orchestrator loop) sits **on top** of this.

# Important notes for this block

* Keep this block **once** in your notebook (avoid duplicates later).
* The base `Environment.execute_action` returns a **uniform envelope**:

  * success → `{"tool_executed": True, "result": ...}`
  * failure → `{"tool_executed": False, "error": "..."}`
    Your ResearchEnvironment will subclass/compose this behavior.
* `ActionRegistry.validate_args` checks **required** params. That’s plenty for now; you can add type checks later if needed.

# What still needs to be defined elsewhere

* **AgentLanguage** (base) and your concrete `SummarizerLanguage`
* **Agent** (the orchestrator loop)
* **ResearchEnvironment** + `make_summarizer` + `build_research_actions`
* The **wiring** for function-calling (`make_generate_response`, goals, agent instantiation)

# Recommended run order (clean, error-free)

1. **STEP 1 — Base Orchestrator (GAME skeleton)** ← this block
2. **Language & Prereqs** (base `AgentLanguage` + `SummarizerLanguage`)
3. **Research Summarizer — Environment & Actions** (ResearchEnvironment, summarizer DI, action registry)
4. **Orchestrator Wiring (function calling)** (OpenAI tools export + `make_generate_response`)
5. **Final wiring** (env, registry, goal, language, agent) + `agent.run(...)`

# One small suggestion

* For max portability (older Python kernels), if you used `int | None` elsewhere, prefer `Optional[int]`.

If you’ve already run this block, you’re set to proceed to Step 2 (Language) and Step 3 (Environment & Actions).


In [ ]:
# STEP 1 — Base Orchestrator (GAME skeleton)
# Run this cell first. It defines the core classes we'll reuse.
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass

# ---- G: Goals --------------------------------------------------------------
@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str

# ---- A: Actions + Registry -------------------------------------------------
class Action:
    def __init__(self, name: str, fn: Callable, description: str, parameters: Dict, terminal: bool=False):
        self.name, self.fn = name, fn
        self.description, self.parameters = description, parameters
        self.terminal = terminal
    def execute(self, **kwargs):
        return self.fn(**kwargs)

class ActionRegistry:
    def __init__(self):
        self._actions: Dict[str, Action] = {}
    def register(self, action: Action):
        if action.name in self._actions:
            raise ValueError(f"Action already registered: {action.name}")
        self._actions[action.name] = action
    def get_action(self, name: str) -> Optional[Action]:
        return self._actions.get(name)
    def get_actions(self) -> List[Action]:
        return list(self._actions.values())
    def validate_args(self, action: Action, args: Dict[str, Any]) -> (bool, str):
        schema = action.parameters or {"type":"object","properties":{},"required":[]}
        for key in schema.get("required", []):
            if key not in args:
                return False, f"Missing required arg: {key}"
        return True, "ok"

# ---- M: Memory -------------------------------------------------------------
class Memory:
    def __init__(self):
        self.items: List[Dict[str, Any]] = []  # each item: {role, content}
    def add_memory(self, m: Dict[str, Any]):
        self.items.append(m)
    def get_memories(self, limit: Optional[int]=None) -> List[Dict[str, Any]]:
        return self.items[-limit:] if limit else self.items

# ---- E: Environment --------------------------------------------------------
class Environment:
    def execute_action(self, action: Action, args: Dict[str, Any]) -> Dict[str, Any]:
        try:
            result = action.execute(**args)
            return {"tool_executed": True, "result": result}
        except Exception as e:
            return {"tool_executed": False, "error": str(e)}

This block is your **orchestrator + base AgentLanguage + smoke test**. It’s correct and useful. A few quick notes so it plays nicely with the Summarizer agent you’re building:

# What this block is for

* Defines a minimal **AgentLanguage** and the **Agent** loop (orchestrator).
* Includes a **hello\_tool** smoke test to prove the loop works without OpenAI/files.

# Keep it — but watch these gotchas

1. **Don’t redefine classes**
   If you already defined `AgentLanguage`, `Agent`, `Action`, `ActionRegistry`, `Memory`, `Environment` earlier, avoid duplicate definitions. Keep *one* canonical version.

2. **Notebook guard**
   In Colab/Jupyter, `if __name__ == "__main__":` won’t fire. If you want the smoke test to run, call it directly (or remove the guard). Not necessary once you move to the summarizer.

3. **Roles & envelopes**
   You’re already writing tool results back to memory as `{"role":"tool", "content": ...}` and using the uniform `{"tool_executed": True|False, ...}` envelope—great. Keep that invariant.

4. **Validation path**
   You validate required args before executing—perfect. If you later add type checks, do it in `ActionRegistry.validate_args`.

# How to switch from smoke test → Summarizer

Leave the orchestrator as-is. Replace the smoke test section with your Research Summarizer wiring (the blocks you have already):

1. Build env + actions:

```python
env = ResearchEnvironment()

def openai_chat_fn(messages):
    resp = client.chat.completions.create(model=MODEL, messages=messages)
    return resp.choices[0].message.content

summarizer = make_summarizer(openai_chat_fn)
registry = build_research_actions(env, summarizer)
```

2. Language + generate\_response (function calling):

```python
language = SummarizerLanguage()
generate_response = make_generate_response(registry)
```

3. Goal + agent + run:

```python
file_summary_goal = Goal(
    priority=1,
    name="file_summary",
    description=(
        "Summarize key points of text documents in /content/files.\n"
        "Steps: 1) list files, 2) read each file, 3) summarize to ≤5 bullets, 4) write to /content/summaries."
    ),
)

agent = Agent(
    goals=[file_summary_goal],
    agent_language=language,
    action_registry=registry,
    generate_response=generate_response,
    environment=env,
)

memory = agent.run("Please summarize all .txt files in /content/files", max_iterations=8, verbose=True)
print("\nTail of memory:", memory.get_memories(4))
```




In [ ]:


# ---- AgentLanguage (prompt builder + parser) ------------------------------
class AgentLanguage:
    def construct_prompt(self, actions: List[Action], environment: Environment, goals: List[Goal], memory: Memory) -> Dict[str, Any]:
        return {
            "goals": [g.description for g in sorted(goals, key=lambda g: g.priority)],
            "tools": [a.name for a in actions],
            "memory": memory.get_memories(6),
        }
    def parse_response(self, response: Dict[str, Any]) -> Dict[str, Any]:
        # Expect a structured dict: {"tool": name, "args": {...}}
        return response

# ---- Orchestrator (Agent) -------------------------------------------------
class Agent:
    def __init__(self, goals, agent_language, action_registry, generate_response, environment):
        self.goals = goals
        self.agent_language = agent_language
        self.actions = action_registry
        self.generate_response = generate_response  # Callable[prompt_dict] -> {tool,args}
        self.environment = environment

    def construct_prompt(self, goals, memory, actions):
        return self.agent_language.construct_prompt(actions=actions.get_actions(),
                                                    environment=self.environment,
                                                    goals=goals,
                                                    memory=memory)

    def prompt_llm_for_action(self, full_prompt):
        return self.generate_response(full_prompt)

    def get_action(self, response):
        invocation = self.agent_language.parse_response(response)
        action = self.actions.get_action(invocation.get("tool"))
        return action, invocation

    def should_terminate(self, response):
        action_def, _ = self.get_action(response)
        return bool(action_def and action_def.terminal)

    def run(self, user_input: str, memory: Optional[Memory]=None, max_iterations: int=3, verbose: bool=True) -> Memory:
        memory = memory or Memory()
        memory.add_memory({"role": "user", "content": user_input})
        for _ in range(max_iterations):
            prompt = self.construct_prompt(self.goals, memory, self.actions)
            if verbose:
                print("Prompt →", prompt)
            response = self.prompt_llm_for_action(prompt)
            if verbose:
                print("Decision ←", response)
            action, invocation = self.get_action(response)
            if not action:
                err = {"tool_executed": False, "error": f"Unknown action: {invocation.get('tool')}"}
                memory.add_memory({"role": "tool", "content": err})
                break
            ok, msg = self.actions.validate_args(action, invocation.get("args", {}))
            if not ok:
                err = {"tool_executed": False, "error": f"Invalid args: {msg}"}
                memory.add_memory({"role": "tool", "content": err})
                continue
            result = self.environment.execute_action(action, invocation.get("args", {}))
            if verbose:
                print("Result ←", result)
            memory.add_memory({"role": "tool", "content": result})
            if not result.get("tool_executed", False):
                memory.add_memory({"role": "assistant", "content": "Got an error; choosing another action next."})
                continue
            if self.should_terminate(response):
                if verbose:
                    print("Terminate signal: stopping loop.")
                break
        return memory

# ---- Smoke test (no OpenAI, no files) -------------------------------------
# Define a tiny tool and a mock "LLM" that always selects it

def hello_tool(name: str = "world"):
    return f"hello, {name}!"

reg = ActionRegistry()
reg.register(Action(
    name="hello_tool",
    fn=hello_tool,
    description="Say hello",
    parameters={"type":"object","properties":{"name":{"type":"string"}},"required":[]}
))

lang = AgentLanguage()

def mock_generate_response(prompt_dict: Dict[str, Any]) -> Dict[str, Any]:
    # Always choose hello_tool with no args
    return {"tool": "hello_tool", "args": {}}

env = Environment()
goals = [Goal(1, "demo", "Run a single tool to confirm wiring works.")]

if __name__ == "__main__":
    agent = Agent(goals, lang, reg, mock_generate_response, env)
    _ = agent.run("Say hi", verbose=True)


Prompt → {'goals': ['Run a single tool to confirm wiring works.'], 'tools': ['hello_tool'], 'memory': [{'role': 'user', 'content': 'Say hi'}]}
Decision ← {'tool': 'hello_tool', 'args': {}}
Result ← {'tool_executed': True, 'result': 'hello, world!'}
Prompt → {'goals': ['Run a single tool to confirm wiring works.'], 'tools': ['hello_tool'], 'memory': [{'role': 'user', 'content': 'Say hi'}, {'role': 'tool', 'content': {'tool_executed': True, 'result': 'hello, world!'}}]}
Decision ← {'tool': 'hello_tool', 'args': {}}
Result ← {'tool_executed': True, 'result': 'hello, world!'}
Prompt → {'goals': ['Run a single tool to confirm wiring works.'], 'tools': ['hello_tool'], 'memory': [{'role': 'user', 'content': 'Say hi'}, {'role': 'tool', 'content': {'tool_executed': True, 'result': 'hello, world!'}}, {'role': 'tool', 'content': {'tool_executed': True, 'result': 'hello, world!'}}]}
Decision ← {'tool': 'hello_tool', 'args': {}}
Result ← {'tool_executed': True, 'result': 'hello, world!'}


# Full Code Dirty

You’re very close—this is a solid build. I’d make a few housekeeping tweaks so it’s rock-steady and easy to reuse.

# What’s great

* Clean, safe **Environment** with a uniform result envelope.
* Clear **ResearchEnvironment** with path safety + truncation flag.
* **ActionRegistry** + JSON-schema-like parameters and basic validation.
* DI via `make_summarizer(...)` (nice!).
* A compact `SummarizerLanguage` and a function-calling `make_generate_response(...)`.

# Changes I recommend

1. **Deduplicate class definitions (keep one canonical copy).**
   In your file, `Goal`, `Action`, `ActionRegistry`, `Memory`, `Environment`, `AgentLanguage`, and `SummarizerLanguage` appear more than once. In a notebook, re-defining can cause subtle mismatches. Keep one version of each.

2. **Order of cells (avoid NameError):**

   * Step 1: GAME skeleton — `Goal`, `Action`, `ActionRegistry`, `Memory`, `Environment`, `AgentLanguage`, `Agent`
   * Step 2: Research layer — `ResearchEnvironment`, `make_summarizer`, `build_research_actions`
   * Step 3: Wiring — `registry_to_openai_tools`, `SummarizerLanguage` (only once), `make_generate_response`, OpenAI client, env, registry, goal, agent, `agent.run(...)`

3. **Use one `SummarizerLanguage`.**
   You define it twice. Keep the more complete one and delete the duplicate.

4. **Standardize on `role` (not `type`) in memory.**
   You sometimes use `{"type": ...}`. Pick `{"role": ...}` everywhere to keep prompts consistent (your wiring already assumes `role` and falls back to `type`).

   **Tiny patch (where you add to memory):**

   ```python
   memory.add_memory({"role": "user", "content": user_input})
   # later
   memory.add_memory({"role": "assistant", "content": response})
   memory.add_memory({"role": "tool", "content": result})
   ```

5. **(Optional, nice) Add a `terminate` tool.**
   Lets the model end cleanly when summaries are saved.

   ```python
   registry.register(Action(
       name="terminate",
       fn=lambda message="All summaries written.": {"message": message},
       description="Stop the loop when work is complete.",
       parameters={"type":"object","properties":{"message":{"type":"string"}}, "required":[]},
       terminal=True,
   ))
   ```

   Then add a gentle hint in your system text (already there) that after saving all summaries, call `terminate`.

6. **Portability nit:** If you ever run on older Python, change `int | None` to `Optional[int]`.

7. **Avoid redefining `AgentLanguage` in the wiring cell.**
   You already defined a base earlier—no need to redeclare. Keep the one base and one `SummarizerLanguage`.

# Minimal “diff” style patches

**A. Use only one `SummarizerLanguage` (keep this one):**

```python
class SummarizerLanguage(AgentLanguage):
    def construct_prompt(self, actions, environment, goals: List[Goal], memory: Memory) -> Dict[str, Any]:
        goals_text = (
            "You are a file summarizer. Follow these goals in order of priority:\n" +
            "\n".join(f"- ({g.priority}) {g.name}: {g.description.strip()}" for g in sorted(goals, key=lambda g: g.priority))
        )
        mem = memory.get_memories(8)
        return {"goals_text": goals_text, "memory": mem, "actions": actions}
```

**B. Standardize memory writes (replace any `type` with `role`):**

```python
memory.add_memory({"role": "user", "content": user_input})
# ...
memory.add_memory({"role": "tool", "content": result})
# (and if you log the agent’s decision)
memory.add_memory({"role": "assistant", "content": response})
```

**C. Add a terminate action (optional, but recommended):**

```python
registry.register(Action(
    name="terminate",
    fn=lambda message="All summaries written.": {"message": message},
    description="Signal that the task is complete.",
    parameters={"type":"object","properties":{"message":{"type":"string"}}, "required":[]},
    terminal=True
))
```

# Quick sanity checklist (before running)

* [ ] Only one copy of each base class exists in the session.
* [ ] The GAME skeleton cell ran **before** research + wiring cells.
* [ ] `OPENAI_API_KEY` is loaded from `/content/API_KEYS.env`.
* [ ] `/content/files` has `.txt` docs, and `/content/summaries` is writeable.
* [ ] (Optional) `terminate` tool is in the registry.

If you want, I can paste a **compact “final order” notebook index** tailored to what you already have, so you can re-run cleanly from top to bottom without duplicate defs.



In [ ]:

# Research Summarizer — Environment & Actions
# - Local text file environment (list/read/write)
# - LLM-powered summarize_text action (uses OpenAI via a small helper)
# - ActionRegistry wiring with JSON-schema-like parameter specs

from typing import Dict, Any, List, Optional, Callable
import os, io, time, traceback, re

# --- Base Environment from your template (kept for consistency) ---
class Environment:
    def execute_action(self, action, args: Dict[str, Any]) -> Dict[str, Any]:
        try:
            result = action.execute(**args)
            return {"tool_executed": True, "result": result, "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")}
        except Exception as e:
            return {
                "tool_executed": False,
                "error": str(e),
                "traceback": traceback.format_exc(),
                "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")
            }

# --- Actions & Registry (same interface as your template) ---
class Action:
    def __init__(self, name: str, fn: Callable, description: str, parameters: Dict, terminal: bool=False):
        self.name, self.fn, self.description = name, fn, description
        self.parameters, self.terminal = parameters, terminal
    def execute(self, **kwargs):
        return self.fn(**kwargs)

class ActionRegistry:
    def __init__(self):
        self._actions: Dict[str, Action] = {}
    def register(self, action: Action):
        if action.name in self._actions:
            raise ValueError(f"Action already registered: {action.name}")
        self._actions[action.name] = action
    def get_action(self, name: str) -> Optional[Action]:
        return self._actions.get(name)
    def get_actions(self) -> List[Action]:
        return list(self._actions.values())
    def validate_args(self, action: Action, args: Dict[str, Any]) -> (bool, str):
        schema = action.parameters or {"type":"object","properties":{},"required":[]}
        for key in schema.get("required", []):
            if key not in args:
                return False, f"Missing required arg: {key}"
        return True, "ok"


# --- Research Summarizer Environment ---
class ResearchEnvironment(Environment):
    base_dir = "/content/files"
    out_dir  = "/content/summaries"

    @staticmethod
    def _safe_join(base: str, name: str) -> str:
        path = os.path.abspath(os.path.join(base, name))
        if not path.startswith(os.path.abspath(base)):
            raise ValueError("Invalid path")
        return path

    @staticmethod
    def _sanitize(name: str) -> str:
        stem = os.path.splitext(os.path.basename(name))[0]
        safe = re.sub(r"[^a-zA-Z0-9._-]", "_", stem)
        return safe + ".summary.txt"

    def list_txt_files(self) -> List[str]:
        if not os.path.exists(self.base_dir):
            return []
        return sorted([f for f in os.listdir(self.base_dir) if f.lower().endswith('.txt')])

    def read_txt_file(self, file_name: str) -> Dict[str, Any]:
        path = self._safe_join(self.base_dir, file_name)
        if not os.path.exists(path):
            return {"error": f"File not found: {file_name}", "hint": "Call list_txt_files first"}
        with io.open(path, 'r', encoding='utf-8', errors='replace') as f:
            text = f.read()
        # Truncate very large documents for safety; agent can iterate in chunks if needed
        max_chars = 12000
        truncated = len(text) > max_chars
        if truncated:
            text = text[:max_chars] + "\n... [truncated]"
        return {"file_name": file_name, "content": text, "truncated": truncated}

    def write_summary_file(self, source_file: str, content: str) -> str:
        os.makedirs(self.out_dir, exist_ok=True)
        out_name = self._sanitize(source_file)
        out_path = self._safe_join(self.out_dir, out_name)
        with io.open(out_path, 'w', encoding='utf-8') as f:
            f.write(content)
        return out_path



# --- LLM Summarization Helper (uses your OpenAI client outside the loop) ---
def make_summarizer(openai_chat_fn: Callable[[List[Dict[str, str]]], str]):
    """Return a summarize_text(text, max_points, style) function using provided LLM call.
    openai_chat_fn: function that takes messages=[...] and returns string content.
    """
    def summarize_text(text: str, max_points: int = 5, style: str = "bullet") -> str:
        system = (
            "You are a precise technical summarizer. Extract key points, preserve facts, "
            "and avoid speculation. Keep it concise."
        )
        user = (
            f"Summarize the following text into at most {max_points} key points. "
            f"Format: {'bullets' if style=='bullet' else 'short paragraphs'}.\n\n" + text
        )
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ]
        return openai_chat_fn(messages)
    return summarize_text

# --- Wiring helper to build the registry for this agent ---
def build_research_actions(env: ResearchEnvironment, summarizer_fn: Callable[[str, int, str], str]) -> ActionRegistry:
    registry = ActionRegistry()

    registry.register(Action(
        name="list_txt_files",
        fn=lambda: env.list_txt_files(),
        description="Return .txt file names from /content/files",
        parameters={"type":"object","properties":{},"required":[]},
    ))

    registry.register(Action(
        name="read_txt_file",
        fn=lambda file_name: env.read_txt_file(file_name),
        description="Read a text file from /content/files",
        parameters={
            "type": "object",
            "properties": {"file_name": {"type": "string"}},
            "required": ["file_name"],
        },
    ))

    registry.register(Action(
        name="summarize_text",
        fn=lambda text, max_points=5, style="bullet": summarizer_fn(text, max_points, style),
        description="Summarize raw text into key points using the LLM",
        parameters={
            "type": "object",
            "properties": {
                "text": {"type": "string"},
                "max_points": {"type": "integer", "minimum": 1, "maximum": 12},
                "style": {"type": "string", "enum": ["bullet", "paragraph"]},
            },
            "required": ["text"],
        },
    ))

    registry.register(Action(
        name="write_summary_file",
        fn=lambda source_file, content: env.write_summary_file(source_file, content),
        description="Write summary text to /content/summaries (auto-named from source)",
        parameters={
            "type": "object",
            "properties": {
                "source_file": {"type": "string"},
                "content": {"type": "string"},
            },
            "required": ["source_file", "content"],
        },
    ))

    return registry


# STEP 2 — Language & Prereqs (clean)
# Put this ABOVE the wiring cell. Defines: Goal, Memory, AgentLanguage, SummarizerLanguage.
from dataclasses import dataclass
from typing import List, Dict, Any

# --- Minimal prereqs --------------------------------------------------------
@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str

class Memory:
    def __init__(self):
        self.items: List[Dict[str, Any]] = []
    def add_memory(self, m: Dict[str, Any]):
        self.items.append(m)
    def get_memories(self, limit: int | None = None):
        return self.items[-limit:] if limit else self.items

# --- AgentLanguage base + concrete SummarizerLanguage ----------------------
class AgentLanguage:
    """Build prompt for the LLM; parse the LLM's response (usually handled by generate_response)."""
    def construct_prompt(self, actions: List[Any], environment: Any, goals: List[Goal], memory: Memory) -> Dict[str, Any]:
        raise NotImplementedError
    def parse_response(self, response: Dict[str, Any]) -> Dict[str, Any]:
        # Default: response already structured as {"tool": ..., "args": {...}}
        return response

class SummarizerLanguage(AgentLanguage):
    """Formats goals/memory for the summarizer agent. Tool-call parsing is done in generate_response()."""
    def construct_prompt(self, actions, environment, goals: List[Goal], memory: Memory) -> Dict[str, Any]:
        goals_text = (
            "You are a file summarizer. Follow these goals in order of priority:\n" +
            "\n".join(f"- ({g.priority}) {g.name}: {g.description.strip()}" for g in sorted(goals, key=lambda g: g.priority))
        )
        mem = memory.get_memories(8)
        return {"goals_text": goals_text, "memory": mem, "actions": actions}

# Research Summarizer — Orchestrator Wiring (function calling)
# REQUIREMENTS (already defined earlier in your notebook):
# - Agent (orchestrator template)
# - Goal, Action, ActionRegistry
# - ResearchEnvironment, make_summarizer, build_research_actions (from the previous cell)
# - Memory class from your template
#
# This cell wires those pieces together, adds an AgentLanguage
# and a generate_response() that uses OpenAI function calling.

import os, json
from typing import Dict, Any, List
from dotenv import load_dotenv
from openai import OpenAI

# ---------------- Load API key & client ----------------
load_dotenv('/content/API_KEYS.env')
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL = "gpt-4o-mini"

# ---------------- Tools export helper ------------------
def registry_to_openai_tools(registry: ActionRegistry) -> List[Dict[str, Any]]:
    tools = []
    for a in registry.get_actions():
        tools.append({
            "type": "function",
            "function": {
                "name": a.name,
                "description": a.description,
                "parameters": a.parameters or {"type": "object", "properties": {}, "required": []},
            },
        })
    return tools

# ---------------- AgentLanguage ------------------------
class SummarizerLanguage(AgentLanguage):
    """Formats goals/memory for the LLM; parse is handled in generate_response."""
    def construct_prompt(self, actions, environment, goals, memory):
        goals_text = "You are a file summarizer. Follow these goals in order of priority:\n" + "\n".join(
            f"- ({g.priority}) {g.name}: {g.description.strip()}" for g in sorted(goals, key=lambda g: g.priority)
        )
        # Keep a tight memory window
        mem = memory.get_memories(8)
        return {"goals_text": goals_text, "memory": mem, "actions": actions}

# ------------- generate_response (OpenAI call) ---------
# NOTE: This returns a structured dict {"tool": name, "args": {...}} for the orchestrator.

def make_generate_response(registry: ActionRegistry):
    tools_spec = registry_to_openai_tools(registry)

    def build_messages(prompt_dict: Dict[str, Any]) -> List[Dict[str, str]]:
        system = (
            prompt_dict["goals_text"]
            + "\n\nYou must use tools via function calling to make progress. "
            + "Choose exactly one next tool per step. If you have saved all summaries, call a terminate tool if available; otherwise indicate completion."
        )
        # Replay memory if you'd like the model to see prior context (optional here)
        memory_msgs = []
        for m in prompt_dict["memory"]:
            role = m.get("role") or m.get("type") or "user"
            content = m.get("content")
            # Coerce non-strings for safety
            if not isinstance(content, str):
                content = json.dumps(content)
            memory_msgs.append({"role": role if role in ("system","user","assistant") else "user", "content": content})

        # Nudge the model with a fresh user instruction
        user_msg = {
            "role": "user",
            "content": (
                "Pick the best next tool from the available functions to progress toward summarizing the files. "
                "Return a function call, not prose."
            ),
        }
        return [{"role": "system", "content": system}] + memory_msgs + [user_msg]

    def _generate_response(prompt_dict: Dict[str, Any]) -> Dict[str, Any]:
        messages = build_messages(prompt_dict)
        resp = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools_spec,
            tool_choice="auto",
            temperature=0.2,
        )
        msg = resp.choices[0].message
        # If the model chose a tool, parse it
        if msg.tool_calls:
            call = msg.tool_calls[0]
            name = call.function.name
            try:
                args = json.loads(call.function.arguments or "{}")
            except json.JSONDecodeError:
                args = {}
            return {"tool": name, "args": args}
        # Fallback if no tool was called; gently kick off with list_txt_files
        return {"tool": "list_txt_files", "args": {}}

    return _generate_response

# ---------------- Build environment & tools -------------
env = ResearchEnvironment()

# Summarizer uses your OpenAI client under the hood

def openai_chat_fn(messages):
    resp = client.chat.completions.create(model=MODEL, messages=messages)
    return resp.choices[0].message.content

summarizer = make_summarizer(openai_chat_fn)
registry = build_research_actions(env, summarizer)

# ---------------- Goals --------------------------------
file_summary_goal = Goal(
    priority=1,
    name="file_summary",
    description=(
        "Summarize key points of text documents in /content/files.\n"
        "Steps: 1) list files, 2) read each file, 3) summarize to ≤5 bullets, 4) write to /content/summaries."
    ),
)

# ---------------- Orchestrator instance -----------------
language = SummarizerLanguage()
generate_response = make_generate_response(registry)

agent = Agent(
    goals=[file_summary_goal],
    agent_language=language,
    action_registry=registry,
    generate_response=generate_response,
    environment=env,
)


# STEP 1 — Base Orchestrator (GAME skeleton)
# Run this cell first. It defines the core classes we'll reuse.
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass

# ---- G: Goals --------------------------------------------------------------
@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str

# ---- A: Actions + Registry -------------------------------------------------
class Action:
    def __init__(self, name: str, fn: Callable, description: str, parameters: Dict, terminal: bool=False):
        self.name, self.fn = name, fn
        self.description, self.parameters = description, parameters
        self.terminal = terminal
    def execute(self, **kwargs):
        return self.fn(**kwargs)

class ActionRegistry:
    def __init__(self):
        self._actions: Dict[str, Action] = {}
    def register(self, action: Action):
        if action.name in self._actions:
            raise ValueError(f"Action already registered: {action.name}")
        self._actions[action.name] = action
    def get_action(self, name: str) -> Optional[Action]:
        return self._actions.get(name)
    def get_actions(self) -> List[Action]:
        return list(self._actions.values())
    def validate_args(self, action: Action, args: Dict[str, Any]) -> (bool, str):
        schema = action.parameters or {"type":"object","properties":{},"required":[]}
        for key in schema.get("required", []):
            if key not in args:
                return False, f"Missing required arg: {key}"
        return True, "ok"

# ---- M: Memory -------------------------------------------------------------
class Memory:
    def __init__(self):
        self.items: List[Dict[str, Any]] = []  # each item: {role, content}
    def add_memory(self, m: Dict[str, Any]):
        self.items.append(m)
    def get_memories(self, limit: Optional[int]=None) -> List[Dict[str, Any]]:
        return self.items[-limit:] if limit else self.items

# ---- E: Environment --------------------------------------------------------
class Environment:
    def execute_action(self, action: Action, args: Dict[str, Any]) -> Dict[str, Any]:
        try:
            result = action.execute(**args)
            return {"tool_executed": True, "result": result}
        except Exception as e:
            return {"tool_executed": False, "error": str(e)}



# ---- AgentLanguage (prompt builder + parser) ------------------------------
class AgentLanguage:
    def construct_prompt(self, actions: List[Action], environment: Environment, goals: List[Goal], memory: Memory) -> Dict[str, Any]:
        return {
            "goals": [g.description for g in sorted(goals, key=lambda g: g.priority)],
            "tools": [a.name for a in actions],
            "memory": memory.get_memories(6),
        }
    def parse_response(self, response: Dict[str, Any]) -> Dict[str, Any]:
        # Expect a structured dict: {"tool": name, "args": {...}}
        return response

# ---- Orchestrator (Agent) -------------------------------------------------
class Agent:
    def __init__(self, goals, agent_language, action_registry, generate_response, environment):
        self.goals = goals
        self.agent_language = agent_language
        self.actions = action_registry
        self.generate_response = generate_response  # Callable[prompt_dict] -> {tool,args}
        self.environment = environment

    def construct_prompt(self, goals, memory, actions):
        return self.agent_language.construct_prompt(actions=actions.get_actions(),
                                                    environment=self.environment,
                                                    goals=goals,
                                                    memory=memory)

    def prompt_llm_for_action(self, full_prompt):
        return self.generate_response(full_prompt)

    def get_action(self, response):
        invocation = self.agent_language.parse_response(response)
        action = self.actions.get_action(invocation.get("tool"))
        return action, invocation

    def should_terminate(self, response):
        action_def, _ = self.get_action(response)
        return bool(action_def and action_def.terminal)

    def run(self, user_input: str, memory: Optional[Memory]=None, max_iterations: int=3, verbose: bool=True) -> Memory:
        memory = memory or Memory()
        memory.add_memory({"role": "user", "content": user_input})
        for _ in range(max_iterations):
            prompt = self.construct_prompt(self.goals, memory, self.actions)
            if verbose:
                print("Prompt →", prompt)
            response = self.prompt_llm_for_action(prompt)
            if verbose:
                print("Decision ←", response)
            action, invocation = self.get_action(response)
            if not action:
                err = {"tool_executed": False, "error": f"Unknown action: {invocation.get('tool')}"}
                memory.add_memory({"role": "tool", "content": err})
                break
            ok, msg = self.actions.validate_args(action, invocation.get("args", {}))
            if not ok:
                err = {"tool_executed": False, "error": f"Invalid args: {msg}"}
                memory.add_memory({"role": "tool", "content": err})
                continue
            result = self.environment.execute_action(action, invocation.get("args", {}))
            if verbose:
                print("Result ←", result)
            memory.add_memory({"role": "tool", "content": result})
            if not result.get("tool_executed", False):
                memory.add_memory({"role": "assistant", "content": "Got an error; choosing another action next."})
                continue
            if self.should_terminate(response):
                if verbose:
                    print("Terminate signal: stopping loop.")
                break
        return memory

# ---- Smoke test (no OpenAI, no files) -------------------------------------
# Define a tiny tool and a mock "LLM" that always selects it

def hello_tool(name: str = "world"):
    return f"hello, {name}!"

reg = ActionRegistry()
reg.register(Action(
    name="hello_tool",
    fn=hello_tool,
    description="Say hello",
    parameters={"type":"object","properties":{"name":{"type":"string"}},"required":[]}
))

lang = AgentLanguage()

def mock_generate_response(prompt_dict: Dict[str, Any]) -> Dict[str, Any]:
    # Always choose hello_tool with no args
    return {"tool": "hello_tool", "args": {}}

env = Environment()
goals = [Goal(1, "demo", "Run a single tool to confirm wiring works.")]

if __name__ == "__main__":
    agent = Agent(goals, lang, reg, mock_generate_response, env)
    _ = agent.run("Say hi", verbose=True)

# Full Code Clean

In [ ]:

# Research Summarizer — Environment & Actions
# - Local text file environment (list/read/write)
# - LLM-powered summarize_text action (uses OpenAI via a small helper)
# - ActionRegistry wiring with JSON-schema-like parameter specs

from typing import Dict, Any, List, Optional, Callable
import os, io, time, traceback, re

# --- Base Environment from your template (kept for consistency) ---
class Environment:
    def execute_action(self, action, args: Dict[str, Any]) -> Dict[str, Any]:
        try:
            result = action.execute(**args)
            return {"tool_executed": True, "result": result, "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")}
        except Exception as e:
            return {
                "tool_executed": False,
                "error": str(e),
                "traceback": traceback.format_exc(),
                "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")
            }

# --- Actions & Registry (same interface as your template) ---
class Action:
    def __init__(self, name: str, fn: Callable, description: str, parameters: Dict, terminal: bool=False):
        self.name, self.fn, self.description = name, fn, description
        self.parameters, self.terminal = parameters, terminal
    def execute(self, **kwargs):
        return self.fn(**kwargs)

class ActionRegistry:
    def __init__(self):
        self._actions: Dict[str, Action] = {}
    def register(self, action: Action):
        if action.name in self._actions:
            raise ValueError(f"Action already registered: {action.name}")
        self._actions[action.name] = action
    def get_action(self, name: str) -> Optional[Action]:
        return self._actions.get(name)
    def get_actions(self) -> List[Action]:
        return list(self._actions.values())
    def validate_args(self, action: Action, args: Dict[str, Any]) -> (bool, str):
        schema = action.parameters or {"type":"object","properties":{},"required":[]}
        for key in schema.get("required", []):
            if key not in args:
                return False, f"Missing required arg: {key}"
        return True, "ok"


# --- Research Summarizer Environment ---
class ResearchEnvironment(Environment):
    base_dir = "/content/files"
    out_dir  = "/content/summaries"

    @staticmethod
    def _safe_join(base: str, name: str) -> str:
        path = os.path.abspath(os.path.join(base, name))
        if not path.startswith(os.path.abspath(base)):
            raise ValueError("Invalid path")
        return path

    @staticmethod
    def _sanitize(name: str) -> str:
        stem = os.path.splitext(os.path.basename(name))[0]
        safe = re.sub(r"[^a-zA-Z0-9._-]", "_", stem)
        return safe + ".summary.txt"

    def list_txt_files(self) -> List[str]:
        if not os.path.exists(self.base_dir):
            return []
        return sorted([f for f in os.listdir(self.base_dir) if f.lower().endswith('.txt')])

    def read_txt_file(self, file_name: str) -> Dict[str, Any]:
        path = self._safe_join(self.base_dir, file_name)
        if not os.path.exists(path):
            return {"error": f"File not found: {file_name}", "hint": "Call list_txt_files first"}
        with io.open(path, 'r', encoding='utf-8', errors='replace') as f:
            text = f.read()
        # Truncate very large documents for safety; agent can iterate in chunks if needed
        max_chars = 12000
        truncated = len(text) > max_chars
        if truncated:
            text = text[:max_chars] + "\n... [truncated]"
        return {"file_name": file_name, "content": text, "truncated": truncated}

    def write_summary_file(self, source_file: str, content: str) -> str:
        os.makedirs(self.out_dir, exist_ok=True)
        out_name = self._sanitize(source_file)
        out_path = self._safe_join(self.out_dir, out_name)
        with io.open(out_path, 'w', encoding='utf-8') as f:
            f.write(content)
        return out_path


In [ ]:



# --- LLM Summarization Helper (uses your OpenAI client outside the loop) ---
def make_summarizer(openai_chat_fn: Callable[[List[Dict[str, str]]], str]):
    """Return a summarize_text(text, max_points, style) function using provided LLM call.
    openai_chat_fn: function that takes messages=[...] and returns string content.
    """
    def summarize_text(text: str, max_points: int = 5, style: str = "bullet") -> str:
        system = (
            "You are a precise technical summarizer. Extract key points, preserve facts, "
            "and avoid speculation. Keep it concise."
        )
        user = (
            f"Summarize the following text into at most {max_points} key points. "
            f"Format: {'bullets' if style=='bullet' else 'short paragraphs'}.\n\n" + text
        )
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ]
        return openai_chat_fn(messages)
    return summarize_text

# --- Wiring helper to build the registry for this agent ---
def build_research_actions(env: ResearchEnvironment, summarizer_fn: Callable[[str, int, str], str]) -> ActionRegistry:
    registry = ActionRegistry()

    registry.register(Action(
        name="list_txt_files",
        fn=lambda: env.list_txt_files(),
        description="Return .txt file names from /content/files",
        parameters={"type":"object","properties":{},"required":[]},
    ))

    registry.register(Action(
        name="read_txt_file",
        fn=lambda file_name: env.read_txt_file(file_name),
        description="Read a text file from /content/files",
        parameters={
            "type": "object",
            "properties": {"file_name": {"type": "string"}},
            "required": ["file_name"],
        },
    ))

    registry.register(Action(
        name="summarize_text",
        fn=lambda text, max_points=5, style="bullet": summarizer_fn(text, max_points, style),
        description="Summarize raw text into key points using the LLM",
        parameters={
            "type": "object",
            "properties": {
                "text": {"type": "string"},
                "max_points": {"type": "integer", "minimum": 1, "maximum": 12},
                "style": {"type": "string", "enum": ["bullet", "paragraph"]},
            },
            "required": ["text"],
        },
    ))

    registry.register(Action(
        name="write_summary_file",
        fn=lambda source_file, content: env.write_summary_file(source_file, content),
        description="Write summary text to /content/summaries (auto-named from source)",
        parameters={
            "type": "object",
            "properties": {
                "source_file": {"type": "string"},
                "content": {"type": "string"},
            },
            "required": ["source_file", "content"],
        },
    ))

    return registry


# STEP 2 — Language & Prereqs (clean)
# Put this ABOVE the wiring cell. Defines: Goal, Memory, AgentLanguage, SummarizerLanguage.
from dataclasses import dataclass
from typing import List, Dict, Any

# --- Minimal prereqs --------------------------------------------------------
@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str

class Memory:
    def __init__(self):
        self.items: List[Dict[str, Any]] = []
    def add_memory(self, m: Dict[str, Any]):
        self.items.append(m)
    def get_memories(self, limit: int | None = None):
        return self.items[-limit:] if limit else self.items

# --- AgentLanguage base + concrete SummarizerLanguage ----------------------
class AgentLanguage:
    """Build prompt for the LLM; parse the LLM's response (usually handled by generate_response)."""
    def construct_prompt(self, actions: List[Any], environment: Any, goals: List[Goal], memory: Memory) -> Dict[str, Any]:
        raise NotImplementedError
    def parse_response(self, response: Dict[str, Any]) -> Dict[str, Any]:
        # Default: response already structured as {"tool": ..., "args": {...}}
        return response

class SummarizerLanguage(AgentLanguage):
    """Formats goals/memory for the summarizer agent. Tool-call parsing is done in generate_response()."""
    def construct_prompt(self, actions, environment, goals: List[Goal], memory: Memory) -> Dict[str, Any]:
        goals_text = (
            "You are a file summarizer. Follow these goals in order of priority:\n" +
            "\n".join(f"- ({g.priority}) {g.name}: {g.description.strip()}" for g in sorted(goals, key=lambda g: g.priority))
        )
        mem = memory.get_memories(8)
        return {"goals_text": goals_text, "memory": mem, "actions": actions}

# Research Summarizer — Orchestrator Wiring (function calling)
# REQUIREMENTS (already defined earlier in your notebook):
# - Agent (orchestrator template)
# - Goal, Action, ActionRegistry
# - ResearchEnvironment, make_summarizer, build_research_actions (from the previous cell)
# - Memory class from your template
#
# This cell wires those pieces together, adds an AgentLanguage
# and a generate_response() that uses OpenAI function calling.

import os, json
from typing import Dict, Any, List
from dotenv import load_dotenv
from openai import OpenAI

# ---------------- Load API key & client ----------------
load_dotenv('/content/API_KEYS.env')
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL = "gpt-4o-mini"

# ---------------- Tools export helper ------------------
def registry_to_openai_tools(registry: ActionRegistry) -> List[Dict[str, Any]]:
    tools = []
    for a in registry.get_actions():
        tools.append({
            "type": "function",
            "function": {
                "name": a.name,
                "description": a.description,
                "parameters": a.parameters or {"type": "object", "properties": {}, "required": []},
            },
        })
    return tools

# ---------------- AgentLanguage ------------------------
class SummarizerLanguage(AgentLanguage):
    """Formats goals/memory for the LLM; parse is handled in generate_response."""
    def construct_prompt(self, actions, environment, goals, memory):
        goals_text = "You are a file summarizer. Follow these goals in order of priority:\n" + "\n".join(
            f"- ({g.priority}) {g.name}: {g.description.strip()}" for g in sorted(goals, key=lambda g: g.priority)
        )
        # Keep a tight memory window
        mem = memory.get_memories(8)
        return {"goals_text": goals_text, "memory": mem, "actions": actions}

# ------------- generate_response (OpenAI call) ---------
# NOTE: This returns a structured dict {"tool": name, "args": {...}} for the orchestrator.

def make_generate_response(registry: ActionRegistry):
    tools_spec = registry_to_openai_tools(registry)

    def build_messages(prompt_dict: Dict[str, Any]) -> List[Dict[str, str]]:
        system = (
            prompt_dict["goals_text"]
            + "\n\nYou must use tools via function calling to make progress. "
            + "Choose exactly one next tool per step. If you have saved all summaries, call a terminate tool if available; otherwise indicate completion."
        )
        # Replay memory if you'd like the model to see prior context (optional here)
        memory_msgs = []
        for m in prompt_dict["memory"]:
            role = m.get("role") or m.get("type") or "user"
            content = m.get("content")
            # Coerce non-strings for safety
            if not isinstance(content, str):
                content = json.dumps(content)
            memory_msgs.append({"role": role if role in ("system","user","assistant") else "user", "content": content})

        # Nudge the model with a fresh user instruction
        user_msg = {
            "role": "user",
            "content": (
                "Pick the best next tool from the available functions to progress toward summarizing the files. "
                "Return a function call, not prose."
            ),
        }
        return [{"role": "system", "content": system}] + memory_msgs + [user_msg]

    def _generate_response(prompt_dict: Dict[str, Any]) -> Dict[str, Any]:
        messages = build_messages(prompt_dict)
        resp = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools_spec,
            tool_choice="auto",
            temperature=0.2,
        )
        msg = resp.choices[0].message
        # If the model chose a tool, parse it
        if msg.tool_calls:
            call = msg.tool_calls[0]
            name = call.function.name
            try:
                args = json.loads(call.function.arguments or "{}")
            except json.JSONDecodeError:
                args = {}
            return {"tool": name, "args": args}
        # Fallback if no tool was called; gently kick off with list_txt_files
        return {"tool": "list_txt_files", "args": {}}

    return _generate_response

# ---------------- Build environment & tools -------------
env = ResearchEnvironment()

# Summarizer uses your OpenAI client under the hood

def openai_chat_fn(messages):
    resp = client.chat.completions.create(model=MODEL, messages=messages)
    return resp.choices[0].message.content

summarizer = make_summarizer(openai_chat_fn)
registry = build_research_actions(env, summarizer)

# ---------------- Goals --------------------------------
file_summary_goal = Goal(
    priority=1,
    name="file_summary",
    description=(
        "Summarize key points of text documents in /content/files.\n"
        "Steps: 1) list files, 2) read each file, 3) summarize to ≤5 bullets, 4) write to /content/summaries."
    ),
)

# ---------------- Orchestrator instance -----------------
language = SummarizerLanguage()
generate_response = make_generate_response(registry)

agent = Agent(
    goals=[file_summary_goal],
    agent_language=language,
    action_registry=registry,
    generate_response=generate_response,
    environment=env,
)


# STEP 1 — Base Orchestrator (GAME skeleton)
# Run this cell first. It defines the core classes we'll reuse.
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass

# ---- G: Goals --------------------------------------------------------------
@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str

# ---- A: Actions + Registry -------------------------------------------------
class Action:
    def __init__(self, name: str, fn: Callable, description: str, parameters: Dict, terminal: bool=False):
        self.name, self.fn = name, fn
        self.description, self.parameters = description, parameters
        self.terminal = terminal
    def execute(self, **kwargs):
        return self.fn(**kwargs)

class ActionRegistry:
    def __init__(self):
        self._actions: Dict[str, Action] = {}
    def register(self, action: Action):
        if action.name in self._actions:
            raise ValueError(f"Action already registered: {action.name}")
        self._actions[action.name] = action
    def get_action(self, name: str) -> Optional[Action]:
        return self._actions.get(name)
    def get_actions(self) -> List[Action]:
        return list(self._actions.values())
    def validate_args(self, action: Action, args: Dict[str, Any]) -> (bool, str):
        schema = action.parameters or {"type":"object","properties":{},"required":[]}
        for key in schema.get("required", []):
            if key not in args:
                return False, f"Missing required arg: {key}"
        return True, "ok"

# ---- M: Memory -------------------------------------------------------------
class Memory:
    def __init__(self):
        self.items: List[Dict[str, Any]] = []  # each item: {role, content}
    def add_memory(self, m: Dict[str, Any]):
        self.items.append(m)
    def get_memories(self, limit: Optional[int]=None) -> List[Dict[str, Any]]:
        return self.items[-limit:] if limit else self.items

# ---- E: Environment --------------------------------------------------------
class Environment:
    def execute_action(self, action: Action, args: Dict[str, Any]) -> Dict[str, Any]:
        try:
            result = action.execute(**args)
            return {"tool_executed": True, "result": result}
        except Exception as e:
            return {"tool_executed": False, "error": str(e)}



# ---- AgentLanguage (prompt builder + parser) ------------------------------
class AgentLanguage:
    def construct_prompt(self, actions: List[Action], environment: Environment, goals: List[Goal], memory: Memory) -> Dict[str, Any]:
        return {
            "goals": [g.description for g in sorted(goals, key=lambda g: g.priority)],
            "tools": [a.name for a in actions],
            "memory": memory.get_memories(6),
        }
    def parse_response(self, response: Dict[str, Any]) -> Dict[str, Any]:
        # Expect a structured dict: {"tool": name, "args": {...}}
        return response

# ---- Orchestrator (Agent) -------------------------------------------------
class Agent:
    def __init__(self, goals, agent_language, action_registry, generate_response, environment):
        self.goals = goals
        self.agent_language = agent_language
        self.actions = action_registry
        self.generate_response = generate_response  # Callable[prompt_dict] -> {tool,args}
        self.environment = environment

    def construct_prompt(self, goals, memory, actions):
        return self.agent_language.construct_prompt(actions=actions.get_actions(),
                                                    environment=self.environment,
                                                    goals=goals,
                                                    memory=memory)

    def prompt_llm_for_action(self, full_prompt):
        return self.generate_response(full_prompt)

    def get_action(self, response):
        invocation = self.agent_language.parse_response(response)
        action = self.actions.get_action(invocation.get("tool"))
        return action, invocation

    def should_terminate(self, response):
        action_def, _ = self.get_action(response)
        return bool(action_def and action_def.terminal)

    def run(self, user_input: str, memory: Optional[Memory]=None, max_iterations: int=3, verbose: bool=True) -> Memory:
        memory = memory or Memory()
        memory.add_memory({"role": "user", "content": user_input})
        for _ in range(max_iterations):
            prompt = self.construct_prompt(self.goals, memory, self.actions)
            if verbose:
                print("Prompt →", prompt)
            response = self.prompt_llm_for_action(prompt)
            if verbose:
                print("Decision ←", response)
            action, invocation = self.get_action(response)
            if not action:
                err = {"tool_executed": False, "error": f"Unknown action: {invocation.get('tool')}"}
                memory.add_memory({"role": "tool", "content": err})
                break
            ok, msg = self.actions.validate_args(action, invocation.get("args", {}))
            if not ok:
                err = {"tool_executed": False, "error": f"Invalid args: {msg}"}
                memory.add_memory({"role": "tool", "content": err})
                continue
            result = self.environment.execute_action(action, invocation.get("args", {}))
            if verbose:
                print("Result ←", result)
            memory.add_memory({"role": "tool", "content": result})
            if not result.get("tool_executed", False):
                memory.add_memory({"role": "assistant", "content": "Got an error; choosing another action next."})
                continue
            if self.should_terminate(response):
                if verbose:
                    print("Terminate signal: stopping loop.")
                break
        return memory

# ---- Smoke test (no OpenAI, no files) -------------------------------------
# Define a tiny tool and a mock "LLM" that always selects it

def hello_tool(name: str = "world"):
    return f"hello, {name}!"

reg = ActionRegistry()
reg.register(Action(
    name="hello_tool",
    fn=hello_tool,
    description="Say hello",
    parameters={"type":"object","properties":{"name":{"type":"string"}},"required":[]}
))

lang = AgentLanguage()

def mock_generate_response(prompt_dict: Dict[str, Any]) -> Dict[str, Any]:
    # Always choose hello_tool with no args
    return {"tool": "hello_tool", "args": {}}

env = Environment()
goals = [Goal(1, "demo", "Run a single tool to confirm wiring works.")]

if __name__ == "__main__":
    agent = Agent(goals, lang, reg, mock_generate_response, env)
    _ = agent.run("Say hi", verbose=True)